In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as transforms
from torchvision import models, datasets
import gensim.downloader
import numpy as np
import sys


In [15]:
def load_data_cifar10(train=True, train_on_embeddings=False):
    transform = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(
                mean=[0.491, 0.482, 0.446],
                std= [0.247, 0.243, 0.261]
            )]) # TODO: Automate calculation given a dataset

    dataset = datasets.CIFAR10(root='/nethome/bdevnani3/raid/data', train=train,
                                            download=True, transform=transform)
    if train_on_embeddings:
        dataset = change_target_to_word_vectors(dataset)

    dataloader = torch.utils.data.DataLoader(dataset, batch_size=4,
                                              shuffle=True, num_workers=2)
    return dataloader

trainloader = load_data_cifar10(True)
testloader = load_data_cifar10(False)

Files already downloaded and verified
Files already downloaded and verified


In [7]:
model = models.resnet18(pretrained=False)
model.fc = nn.Linear(in_features=512, out_features=10)
model.load_state_dict(torch.load("/nethome/bdevnani3/raid/trained_models/vis_lang/pred_class.pt"))
model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [12]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9, weight_decay=5e-4)

In [ ]:
test_loss = 0.0
total  = 0
correct = 0
with torch.no_grad():
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs, labels = inputs.to(device), labels.to(device)

        if torch.cuda.is_available():
            model.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
print("Train Loss: {} | Acc: {} | {}/{}".format(test_loss/len(testloader), 100.*correct/total, correct, total))

In [14]:
test_loss = 0.0
total  = 0
correct = 0
with torch.no_grad():
    for i, data in enumerate(testloader, 0):
        inputs, labels = data
        device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        inputs, labels = inputs.to(device), labels.to(device)

        if torch.cuda.is_available():
            model.cuda()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        test_loss += loss.item()
        
        _, predicted = outputs.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
print("Test Loss: {} | Acc: {} | {}/{}".format(test_loss/len(testloader), 100.*correct/total, correct, total))

Loss: 0.5178078583598137 | Acc: 83.65 | 8365/10000
